In [16]:
import pandas as pd
import numpy as np
from collections import Counter

## Extract data from Excel

In [3]:
df_raw = pd.read_excel('Raw_Data.xlsx')

## Data Exploration

In [15]:
# Load basic stats
basic_info = df_raw.describe(include='all')
missing_counts = df_raw.isnull().sum()
unique_counts = df_raw.nunique()
sample_responses = df_raw.sample(5, random_state=1)

# Prepare a summary dictionary for markdown
exploration_summary = {
    "total_rows": df_raw.shape[0],
    "total_columns": df_raw.shape[1],
    "missing_values": missing_counts.to_dict(),
    "unique_values": unique_counts.to_dict()
}

## 🧭 Initial Data Exploration

Before diving into cleaning, I conducted an initial exploration to understand the structure, quality, and uniqueness of the data. This step ensures we make informed choices in the cleaning phase.

### 📌 Dataset Summary
- **Total Rows:** 47
- **Total Columns:** 3

### 🧼 Missing Values
- **Customer:** 0 missing
- **Check-In Questionnaire:** 0 missing
- **Check-Out Questionnaire:** 5 missing  
  → Indicates that some guests did not leave comments after their stay.

### 🧠 Uniqueness of Responses
- **47 unique customers** (1 per row)
- **44 unique Check-In responses**
- **35 unique Check-Out impressions**

### 🔍 Sample Snapshot
To get a feel for the kind of feedback received, I reviewed a random sample of responses from the raw dataset. This helped shape decisions on text handling and transformation steps used later in data cleaning.


## Data Cleaning

In [25]:
# Rename 'Customer' column to 'CustomerId'
df_cleaned = df_raw.rename(columns={'Customer': 'CustomerId'})

# Replace 'CustomerX' → numeric ID (e.g., 'Customer1' → 1)
df_cleaned['CustomerId'] = df_cleaned['CustomerId'].str.extract('(\d+)').astype(int)
df_cleaned = df_cleaned.rename(columns={
    'Check-In Questionnaire: What persuaded you to choose our hotel for your stay?': 'CheckIn_Reason',
    'Check-Out Questionnaire: What impressed you the most during your stay?': 'CheckOut_Impression'
})

# Define a function to clean semicolon-separated answers into list of phrases
def clean_response_column(col):
    return col.dropna().apply(lambda x: [item.strip() for item in x.split(';')])

# Apply to both textual columns (if not null)
df_cleaned['CheckIn_Reason_List'] = clean_response_column(df_cleaned['CheckIn_Reason'])
df_cleaned['CheckOut_Impression_List'] = clean_response_column(df_cleaned['CheckOut_Impression'])
df_cleaned = df_cleaned.drop(columns=['CheckIn_Reason','CheckOut_Impression'])

# Replace Nan with [] for consistency
df_cleaned['CheckIn_Reason_List'] = df_cleaned['CheckIn_Reason_List'].apply(lambda x: x if isinstance(x, list) else [])
df_cleaned['CheckOut_Impression_List'] = df_cleaned['CheckOut_Impression_List'].apply(lambda x: x if isinstance(x, list) else [])

## 🔍 Summary of Data Cleaning Steps

To ensure accurate and meaningful insights from the hotel feedback data, I cleaned and transformed the dataset to make it analysis-ready:

### ✅ Simplified Customer Information
- Renamed the customer column to **`CustomerId`** for clarity.
- Converted IDs like `"Customer1"`, `"Customer2"` into clean numeric values (e.g., `1`, `2`, `3`), making it easier to manage and track each entry.

### ✅ Clarified Questionnaire Responses
- Renamed long column titles into short, meaningful names:
  - *"What persuaded the guest to choose the hotel"* → **`CheckIn_Reason`**
  - *"What impressed them most after their stay"* → **`CheckOut_Impression`**

### ✅ Structured the Answers for Better Analysis
- Guests often gave multiple reasons separated by semicolons (e.g., `"great service; delicious food"`).
- These responses were split into individual points and stored as **lists**.
- This makes it easier to:
  - Track frequent mentions like “great service”
  - Analyze patterns in expectations vs. experiences

### ✅ Removed Redundant Columns
- After cleaning, the original text columns were removed to keep the dataset **focused and structured**.


In [26]:
df_cleaned

,CustomerId,CheckIn_Reason_List,CheckOut_Impression_List
0,1,"[Delicious Breakfast, Fast and Reliable Wi-Fi,...","[Fast and Reliable Wi-Fi, Delicious Breakfast]"
1,2,"[Quiet and Restful Environment, Fast and Relia...","[Quiet and Restful Environment, Fast and Relia..."
2,3,"[Reservation & Communication, Fast and Reliabl...","[Fast and Reliable Wi-Fi, Reservation & Commun..."
3,4,"[Stylish Interior Design, Quiet and Restful En...","[Quiet and Restful Environment, Friendly and H..."
4,5,"[Stylish Interior Design, Quiet and Restful En...",[Reservation & Communication]
5,6,"[Comfortable and Clean Rooms, Quiet and Restfu...","[Comfortable and Clean Rooms, Quiet and Restfu..."
6,7,"[Comfortable and Clean Rooms, Quiet and Restfu...","[Quiet and Restful Environment, Modern Fitness..."
7,8,"[Comfortable and Clean Rooms, Quiet and Restfu...","[Comfortable and Clean Rooms, Quiet and Restfu..."
8,9,"[Easy Parking & Check-in, Modern Fitness Facil...","[Modern Fitness Facilities, Comfortable and Cl..."
9,10,"[Quiet and Restful Environment, Reservation & ...","[Fast and Reliable Wi-Fi, Reservation & Commun..."


## Data Analysis

In [50]:
# Function to calculate match and extra feedback per person
def calculate_alignment(row):
    checkin_set = set(row['CheckIn_Reason_List'])
    checkout_set = set(row['CheckOut_Impression_List'])

    if not checkin_set:
        return pd.Series([np.nan, np.nan, np.nan])

    matched = len(checkin_set & checkout_set)
    unmatched = len(checkin_set - checkout_set)
    extra = len(checkout_set - checkin_set)
    match_percent = round((matched / len(checkin_set)) * 100, 2)

    return pd.Series([matched, match_percent, extra])

# Apply the function row-wise to the cleaned dataframe
df_cleaned[['Matched_Count', 'Match_Percent', 'Extra_Impressions']] = df_cleaned.apply(calculate_alignment, axis=1)

# Updated version of the label_alignment function to handle empty check-in or check-out lists
def label_alignment(row):
    checkin_list = row['CheckIn_Reason_List']
    checkout_list = row['CheckOut_Impression_List']

    if not checkin_list and not checkout_list:
        return 'No Check-In & No Check-Out Response'
    elif not checkin_list:
        return 'No Check-In Response'
    elif not checkout_list:
        return 'No Check-Out Response'

    checkin_set = set(checkin_list)
    checkout_set = set(checkout_list)
    matched = len(checkin_set & checkout_set)
    extra = len(checkout_set - checkin_set)
    match_percent = round((matched / len(checkin_set)) * 100, 2)

    if match_percent == 100 and extra > 0:
        return 'Fully Matched + Extra'
    elif match_percent == 100:
        return 'Fully Matched'
    elif match_percent == 0:
        return 'No Match'
    elif match_percent <= 20:
        return 'Matched ≤ 20%'
    elif match_percent <= 40:
        return 'Matched ≤ 40%'
    elif match_percent <= 60:
        return 'Matched ≤ 60%'
    elif match_percent <= 80:
        return 'Matched ≤ 80%'
    elif match_percent < 100:
        return 'Matched ≤ 99%'  # Covers anything between 80 and <100

# Apply updated logic
df_cleaned['Alignment_Tag'] = df_cleaned.apply(label_alignment, axis=1)

# Drop intermediate calculation columns if they exist
df_cleaned = df_cleaned.drop(columns=[col for col in ['Matched_Count', 'Match_Percent', 'Extra_Impressions'] if col in df_cleaned.columns])

In [54]:
# Get the count of each new tiered alignment label
alignment_tier_counts = df_cleaned['Alignment_Tag'].value_counts().reset_index()
alignment_tier_counts.columns = ['Alignment Category', 'Count']

# Descriptions for each tier
tier_descriptions = {
    'Fully Matched': 'All expectations were reflected in Check-Out feedback',
    'Fully Matched + Extra': 'All expectations matched, plus unprompted positive surprises',
    'Matched ≤ 99%': 'Matched nearly all expectations, but not all',
    'Matched ≤ 80%': 'Met a large majority of expectations',
    'Matched ≤ 60%': 'Met more than half of expectations',
    'Matched ≤ 40%': 'Met some expectations',
    'Matched ≤ 20%': 'Met very few expectations',
    'No Match': 'None of the stated expectations were mentioned afterward',
    'No Check-Out Response': 'Guest did not leave post-stay feedback',
    'No Check-In Response': 'Guest did not state any initial expectations',
    'No Check-In & No Check-Out Response': 'Guest provided no feedback at all'
}

# Map tier descriptions
alignment_tier_counts['Description'] = alignment_tier_counts['Alignment Category'].map(tier_descriptions)

# Reorder columns
alignment_tier_counts = alignment_tier_counts[['Alignment Category', 'Description', 'Count']]


In [55]:
alignment_tier_counts

,Alignment Category,Description,Count
0,Matched ≤ 80%,Met a large majority of expectations,15
1,Matched ≤ 60%,Met more than half of expectations,11
2,Fully Matched,All expectations were reflected in Check-Out f...,8
3,Matched ≤ 40%,Met some expectations,6
4,No Check-Out Response,Guest did not leave post-stay feedback,5
5,Matched ≤ 20%,Met very few expectations,1
6,Matched ≤ 99%,"Matched nearly all expectations, but not all",1


## 💡 Deeper Guest Alignment Insights (Tiered Analysis)

To assess how well the hotel delivered on individual guest expectations, we performed a person-level comparison of each guest’s Check-In reasons with their Check-Out impressions. This helped quantify not just whether expectations were met — but **to what extent** they were fulfilled.

---

### 📊 Alignment Breakdown

| Alignment Category        | Description                                        | Count |
|---------------------------|----------------------------------------------------|-------|
| Matched ≤ 80%             | Met a large majority of expectations               | 15    |
| Matched ≤ 60%             | Met more than half of expectations                 | 11    |
| Fully Matched             | All expectations were reflected in Check-Out feedback | 8  |
| Matched ≤ 40%             | Met some expectations                              | 6     |
| No Check-Out Response     | Guest did not leave post-stay feedback             | 5     |
| Matched ≤ 20%             | Met very few expectations                          | 1     |
| Matched ≤ 99%             | Matched nearly all expectations, but not all       | 1     |

---

### 🧠 What the Hotel Can Learn

#### 1. Precision Delivery Still Has Gaps  
Only 8 out of 47 guests reported **complete alignment** between expectations and experience. This reveals **inconsistency in execution** — even among guests who received mostly what they asked for.

#### 2. Over Half of Guests Received Partial Satisfaction  
The majority of guests fall into the **40–80% match range**, meaning they noticed a meaningful gap in delivery. While they may still be satisfied, **these guests are less likely to become loyal or refer others**.

#### 3. Missed Engagement with Some Guests  
5 guests did not leave any Check-Out feedback — a **10% insight loss**. Whether due to timing, survey format, or disengagement, this indicates a need for a better **feedback loop**.

---

### 🔧 Strategic Recommendations

| Focus Area           | Action Step                                                                 |
|----------------------|------------------------------------------------------------------------------|
| **Operational Focus** | Identify and strengthen weaker service categories that frequently go unmatched |
| **Tier-Based Service Reviews** | Use these match tiers in staff reviews to drive accountability by theme |
| **Feedback Completion** | Implement reminder nudges or rewards to improve Check-Out response rates |
| **Recovery Strategy** | Proactively follow up with guests whose alignment falls below 40%            |

---

### 🎯 Bottom Line

This tiered alignment model goes beyond general feedback trends and shows **how well the hotel is delivering per guest**. The findings suggest:
- **Solid coverage of expectations**, but not with enough consistency.
- A need for **better feedback capture** and **theme-specific service assurance**.

With targeted improvements, the hotel can boost full alignment rates — a strong driver of retention, referrals, and reputation.